## Prerequisites

In [1]:
import os
from os import chdir

import pandas as pd 
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from scipy.sparse import load_npz
from scipy.sparse import save_npz

chdir(
    r'C:\Users\laplace-transform\AppData\Local\Programs\Python\Python37\notebooks\2020-knu-ai-master\jigsaw-toxic-comment-classification-challenge'
)

### Note! Some of these models support only multiclass classification, please, while selecting your dataset,  
### be sure that for algorithms which does not support multilabel classification you use only examples with only one label. 
### Examples without a label in any of the provided categories are clean messages, without any toxicity.

In [2]:
df = pd.read_csv("../jigsaw-toxic-comment-classification-challenge/train.csv")

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.shape

(159571, 8)

### As one of the methods to make the training simpier, use only examples, assigned to any category vs clean examples.  
For example:  
- Select only messages with obscene label == 1  
- Select all of the "clean" messages  
Implement a model which can perform a binary classification  - to understand whether your message is obscene or not.   

##### If you want to perform a multilabel classification, please understand the difference between multilabel and multiclass classification and be sure that you are solving the correct task - choose only algorithms applicable for solving this type of problem.

#### To work with multiclass task:  
You only need to select messages which have only one label assigned: message cannot be assigned to 2 or more categories.  

#### To work with multilabel task: 
You can work with the whole dataset - some of your messages have only 1 label, some more than 1. 

## Text vectorization

Previously we worked only with words vectorization. But we need to have a vector for each text, not only words from it. 

Before starting a text vectorization, please, make sure you are working with clean data - use the dataset created on the previous day. Cleaned from punctuation, stop words, lemmatized or stemmed, etc. 

In [5]:
from string import punctuation

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words('english'))

In [6]:
def preprocess_text(tokenizer, lemmatizer, stop_words, punctuation, text): 
    tokens = tokenizer(text.lower())
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return [token for token in lemmas if token not in stop_words and token not in punctuation]

df['cleaned'] = df.comment_text.apply(lambda x: preprocess_text(word_tokenize, lemmatizer, stop_words, punctuation, x))

In [7]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[d'aww, match, background, colour, 'm, seeming..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, 'm, really, trying, edit, war, 's, ..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[``, ca, n't, make, real, suggestion, improvem..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page, 's]"


In [8]:
def flat_nested(nested):
    flatten = []
    for item in nested:
        if isinstance(item, list):
            flatten.extend(item)
        else:
            flatten.append(item)
    return flatten

In [9]:
vocab = set(flat_nested(df.cleaned.tolist()))

In [10]:
len(vocab)

249531

As we see, probably you vocabulary is too large.  
Let's try to make it smaller.  
For example, let's get rig of words, which has counts in our dataset less than some threshold.

In [11]:
from collections import Counter, defaultdict 

cnt_vocab = Counter(flat_nested(df.cleaned.tolist()))

In [12]:
cnt_vocab.most_common(10)

[("''", 241319),
 ('``', 156982),
 ('article', 73264),
 ("'s", 66766),
 ("n't", 57144),
 ('wa', 56590),
 ('page', 56239),
 ('wikipedia', 45413),
 ('talk', 35356),
 ('ha', 31896)]

You can clean words which are shorter that particular length and occur less than N times. 

In [13]:
threshold_count = 10
threshold_len = 4 
cleaned_vocab = [token for token, count in cnt_vocab.items() if count > threshold_count and len(token) > threshold_len]

In [14]:
len(cleaned_vocab)

18705

Much better!  
Let's try to vectorize the text summing one-hot vectors for each word. 

In [15]:
vocabulary = defaultdict()

for i, token in enumerate(cleaned_vocab): 
    empty_vec = np.zeros(len(cleaned_vocab))
    empty_vec[i] = 1 
    vocabulary[token] = empty_vec

In [16]:
vocabulary['hardcore']

array([0., 0., 0., ..., 0., 0., 0.])

Rigth now we have vectors for words (words are one-hot vectorized)  
Let's try to create vectors for texts: 

In [17]:
sample_text = df.cleaned[10]
print(sample_text)

['``', 'fair', 'use', 'rationale', 'image', 'wonju.jpg', 'thanks', 'uploading', 'image', 'wonju.jpg', 'notice', 'image', 'page', 'specifies', 'image', 'used', 'fair', 'use', 'explanation', 'rationale', 'use', 'wikipedia', 'article', 'constitutes', 'fair', 'use', 'addition', 'boilerplate', 'fair', 'use', 'template', 'must', 'also', 'write', 'image', 'description', 'page', 'specific', 'explanation', 'rationale', 'using', 'image', 'article', 'consistent', 'fair', 'use', 'please', 'go', 'image', 'description', 'page', 'edit', 'include', 'fair', 'use', 'rationale', 'uploaded', 'fair', 'use', 'medium', 'consider', 'checking', 'specified', 'fair', 'use', 'rationale', 'page', 'find', 'list', "'image", 'page', 'edited', 'clicking', '``', "''", 'contribution', "''", "''", 'link', 'located', 'top', 'wikipedia', 'page', 'logged', 'selecting', '``', "''", 'image', "''", "''", 'dropdown', 'box', 'note', 'fair', 'use', 'image', 'uploaded', '4', 'may', '2006', 'lacking', 'explanation', 'deleted', 'one

### One-hot vectorization and count vectorization

In [18]:
sample_vector = np.zeros(len(cleaned_vocab))

for token in sample_text: 
    try: 
        sample_vector += vocabulary[token]
    except KeyError: 

        continue

In [19]:
sample_vector

array([3., 0., 0., ..., 0., 0., 0.])

Right now we have count vectorization for our text.   
Use this pipeline to create vectors for all of the texts. Save them into np.array. i-th raw in np.array is a vector which represents i-th text from the dataframe.  

In [20]:
from scipy.sparse import csr_matrix

def vocabulary_interact_vect(
    sample_text:     np.ndarray,
    vocabulary_len:  int,
    vect_vocabulary: defaultdict):
    text_vector = np.zeros(len(vect_vocabulary))
    for token in sample_text:
        try: 
            text_vector += vect_vocabulary[token]
        except KeyError: 
            continue
    return text_vector


# because we're going to use matrix interpret on train/test data
# by the way, this algorithm is pretty slow, but gives correct results
def vocabulary_interact_sparse(
    corpus:          pd.Series,
    vocabulary_len:  int,
    vect_vocabulary: defaultdict):
    corpus_len = len(corpus)
    texts_vectorized = csr_matrix((corpus_len, vocabulary_len))
    for j in range(corpus_len):
        current_text_vector = vocabulary_interact_vect(
            corpus[j], vocabulary_len, vect_vocabulary
        )
        
        current_sparse_matr = csr_matrix(
            (current_text_vector,(np.full(N, j),np.arange(N))), 
            shape = (corpus_len, vocabulary_len)
        )
        
        texts_vectorized += current_sparse_matr
    
    return texts_vectorized

In [21]:
N = len(cleaned_vocab)

text_vectorized = vocabulary_interact_sparse(
    corpus = pd.Series([df.cleaned[10]]), vocabulary_len = N, vect_vocabulary = vocabulary
)

text_vectorized.toarray()

array([[3., 0., 0., ..., 0., 0., 0.]])

In [22]:
util_load = True

path0 = os.getcwd() + os.sep + 'document_term_matrix.npz'

if util_load:
    text_vectorized = load_npz(path0)
else:
    text_vectorized = vocabulary_interact_sparse(
        corpus = df.cleaned, vocabulary_len = N, vect_vocabulary = vocabulary
    )

In [23]:
text_vectorized

<159571x18705 sparse matrix of type '<class 'numpy.float64'>'
	with 2717363 stored elements in Compressed Sparse Row format>

In [24]:
util_save = False

if util_save:
    save_npz(path0, text_vectorized)
    
util_del = True

if util_del:
    del text_vectorized

### The next step is to train any classification model on top of the received vectors and report the quality. 

Please, select any of the proposed pipelines for performing a text classification task. (Binary, multiclass or multilabel).  

The main task to calculate our models performance is to create a training and test sets. When you selected a texts for your task, please, use https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html to have at least two sets - train and test.  

Train examples you will use to train your model on and test examples to evaluate your model - to understand how your model works on the unseen data. 

### Train-test split 

In [25]:
### Your code here, splitting your dataset into train and test parts.

# This time i'm going to handle binary classification task
# First of all, I'll divide df into two separate groups:
# - non-toxic data
# - insult labelled data

df_categories = [
    'identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic'
]

crucial_data = df[[df_categories[1], 'cleaned']]

df_non_toxic = crucial_data[~df[df_categories].any(axis = 'columns')]
df_insulting = crucial_data[df.insult != 0]

df_combined = df_non_toxic.append(df_insulting).reset_index(drop = True)
df_combined.columns = ['label', 'texts']

print(
    df_combined.head(),
    df_combined.tail(),
    sep = '\n\n'
)

   label                                              texts
0      0  [explanation, edits, made, username, hardcore,...
1      0  [d'aww, match, background, colour, 'm, seeming...
2      0  [hey, man, 'm, really, trying, edit, war, 's, ...
3      0  [``, ca, n't, make, real, suggestion, improvem...
4      0            [sir, hero, chance, remember, page, 's]

        label                                              texts
151218      1  [``, previous, conversation, fucking, shit, ea...
151219      1                        [mischievious, pubic, hair]
151220      1  [absurd, edits, absurd, edits, great, white, s...
151221      1  [``, hey, listen, n't, ever, delete, edits, ev...
151222      1  ['m, going, keep, posting, stuff, u, deleted, ...


In [26]:
from sklearn.model_selection import train_test_split

custom_test_size = 0.25

X = df_combined['texts']
Y = df_combined['label']

# making train and test sets for future model
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = custom_test_size
)

X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)

Y_train = Y_train.reset_index(drop = True)
Y_test = Y_test.reset_index(drop = True)

In [27]:
# Train set
X_train.head()

0           [block, ever, block, ever, *bangs, table*]
1    [arvanitas, autochthon, karaguni, withour, ‘, ...
2    [french, fry, clue, chip, ..., .., 94.168.193.75]
3    [``, term, ``, '', massive, star, '', '', defi...
4    [``, please, provide, way, explain, look, like...
Name: texts, dtype: object

In [28]:
# Test set
X_test.head()

0    [noticed, commented, article, talk, page, dele...
1                                 [ha, since, removed]
2    [``, ``, '', father, nation, '', '', infobox, ...
3    [``, :i, 'm, curious, sir, fozzie, ``, '', bur...
4    [spot, 'islam, 'feminism, 'veganism, etc., com...
Name: texts, dtype: object

### TF-IDF score 

#### Please, review again this article or read it if you have not done it before. 

https://medium.com/@paritosh_30025/natural-language-processing-text-data-vectorization-af2520529cf7

#### Implement calculating a tf-idf score for each of the words from your vocabulary. 

The main goal of this task is to create a dictionary - keys of the dictionary would be tokens and values would be corresponding tf-idf score of the token.

#### Calculate it MANUALLY and compare the received scores for words with the sklearn implementation:  

#### Tip: 

##### TF = (Number of time the word occurs in the current text) / (Total number of words in the current text)  

##### IDF = (Total number of documents / Number of documents with word t in it)

##### TF-IDF = TF*IDF 

When you calculated a tf-idf score for each of the words in your vocabulary - revectorize the texts.  
Instead of using number of occurences of the i-th word in the i-th cell of the text vector, use it's tf-idf score.   

Revectorize the documents, save vectors into np.array. 

In [29]:
### Your code here for obtaining a tf-idf vectorized documents. 

# Like mentioned above, we're going to define 
# the tf-idf vectorization method manually.

def TF_binary(
    term:     str,       # our token
    document: list):     # our text - a list of cleaned tokens
    return term in document


def TF(
    term:     str,       # our token
    document: list):     # our text - a list of cleaned tokens
    return document.count(term)/len(document) if document else 0


def IDF(
    term:     str,       # our token
    corpus:   pd.Series, # list of all texts, to which the mentioned one belongs
    use_log:  bool):     # apply log func on a result or not
    TF_binary_v_rough = np.vectorize(lambda doc: TF_binary(term, doc))
    return (np.log if use_log else (lambda t: t))(
        len(corpus)/len(corpus[TF_binary_v_rough(corpus)])
    )


def TF_IDF(
    term:     str,       # our token
    document: list,      # our text - a list of cleaned tokens
    corpus:   pd.Series, # list of all texts, to which the mentioned one belongs
    use_log:  bool):     # use IDF or IDF_enchanced in calculations below
    return TF(term, document) * (
        IDF(term, corpus, use_log)
    )

In [30]:
df_sample = df.cleaned[10]
ex_term = 'one'

print(
    "Term:{tm:>10}\n"
    "TF_bin:{tb:>6}\n"
    "TF_std:{ts:>25}\n".format(
        tm = ex_term,
        tb = TF_binary(ex_term, df_sample),
        ts = TF(ex_term, df_sample) # 16/276
    )
)

df_sample_series = df.cleaned[8:13]

print(df_sample_series, end = '\n\n')

print(
    "Term:{tm:>10}\n"
    "IDF_std:{ib:>22}\n"
    "IDF_log:{il:>22}\n".format(
        tm = ex_term,
        ib = IDF(ex_term, df_sample_series, use_log = False),
        il = IDF(ex_term, df_sample_series, use_log = True)
    )
)

print(
    "Term:{tm:>10}\n"
    "TF-IDF_std: {tib}\n"
    "TF-IDF_log: {til}\n".format(
        tm = ex_term,
        tib = TF_IDF(ex_term, df_sample, df_sample_series, use_log = False),
        til = TF_IDF(ex_term, df_sample, df_sample_series, use_log = True)
    )
)

del df_sample_series, df_sample, ex_term

Term:       one
TF_bin:     1
TF_std:     0.014492753623188406

8     [sorry, word, 'nonsense, wa, offensive, anyway...
9             [alignment, subject, contrary, dulithgow]
10    [``, fair, use, rationale, image, wonju.jpg, t...
11             [bbq, man, let, discus, it-maybe, phone]
12    [hey, ..., it.., talk, ..., exclusive, group, ...
Name: cleaned, dtype: object

Term:       one
IDF_std:    1.6666666666666667
IDF_log:    0.5108256237659907

Term:       one
TF-IDF_std: 0.024154589371980676
TF-IDF_log: 0.007403269909652039



In [31]:
from IPython.display import clear_output

In [32]:
# awful. definitely needs proper manual vectorization
def TF_IDF_dict(
    corpus:     pd.Series,   # list of all texts
    vocabulary: defaultdict, # generated set of uniq words
    use_log:    bool):       # use IDF or IDF_enchanced in calculations below
    tf_idf_dict = defaultdict()
    j = 1
    for token in vocabulary:
        clear_output()
        print("{c} | {e}".format(c = j, e = len(vocabulary)))
        tf_idf_dict[token] = np.array(
            [(doc.count(token)/len(doc) if doc else 0) for doc in corpus]
        ) * (np.log if use_log else (lambda t: t))(
        len(corpus)/len(corpus[np.vectorize(lambda doc: token in doc)(corpus)])
        )
        j += 1
        
        
def TF_IDF_sparse(
    corpus:     pd.Series,   # list of all texts
    vocabulary: defaultdict, # generated set of uniq words
    use_log:    bool):       # use IDF or IDF_enchanced in calculations below
    text_vectorized = csr_matrix(len(corpus),len(vocabulary))
    IDF_vectorized = len(corpus) / np.array(
        [
            np.sum(corpus.apply(lambda l: t in l)) for t in vocabulary
        ]
    )
    
    
    

In [33]:
# code for TF_IDF_sparse usage

In [34]:
# no-sklearn implementation

path1 = os.getcwd() + os.sep + 'x_train_tf_idf_sparse.npz'

if util_load:
    X_train_count_matrix = load_npz(path1)
else:
    X_train_count_matrix = vocabulary_interact_sparse(
        corpus = X_train, vocabulary_len = N, vect_vocabulary = vocabulary
    )

X_train_count_matrix

<113417x18705 sparse matrix of type '<class 'numpy.float64'>'
	with 1967295 stored elements in Compressed Sparse Row format>

In [35]:
if util_save:
    
    save_npz(path1, X_train_count_matrix)
    
if util_del:
    
    del X_train_count_matrix

In [37]:
# sklearn implementation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

X_train_concat = X_train.str.join(' ')
X_test_concat = X_test.str.join(' ')

pipeline_global = Pipeline(
    [
        ('cnt-matr', CountVectorizer(vocabulary = cleaned_vocab)),
        ('tf-idf-sparse-matr', TfidfTransformer())
    ]
)

X_train_tf_idf_sparse = pipeline_global.fit_transform(X_train_concat)
X_test_tf_idf_sparse = pipeline_global.transform(X_test_concat)

In [38]:
X_train_tf_idf_sparse

<113417x18705 sparse matrix of type '<class 'numpy.float64'>'
	with 1985383 stored elements in Compressed Sparse Row format>

### Training the model 

As it was said before, select any of the text classification models for the selected task and train the model. 

When the model is trained, you need to evaluate it somehow. 

Read about True positive, False positive, False negative and True negative counts and how to calculate them:   

https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative 

##### Calculate TP, FP, FN and TN on the test set for your model to measure its performance. 


In [39]:
linear_classifier = LogisticRegression(
    random_state = 0                     # <=> seed equiv.
).fit(X_train_tf_idf_sparse, Y_train)

In [40]:
#linear_classifier.score(X_train_tf_idf_sparse, Y)

Y_prediction = linear_classifier.predict(X_test_tf_idf_sparse)

print(Y_prediction.shape)
print(Y_test.shape)

(37806,)
(37806,)


In [41]:
# assuming that we have only 0 or 1 in Y_test/Y_pred

def TP_FP_FN_TN(
    y_test,       # validated labels
    y_pred):      # predicted labels
    TN = lambda y_test, y_pred: np.sum([(y_test==y_pred) & y_test]) 
    TP = lambda y_test, y_pred: np.sum([(y_test==y_pred) & ~y_test])
    
    FN = lambda y_test, y_pred: np.sum([(y_test!=y_pred) & y_test])
    FP = lambda y_test, y_pred: np.sum([~((y_test==y_pred) | y_test)])
    
    return np.array(list(map(lambda f: f(y_test, y_pred),[TP, FP, FN, TN]))).reshape((2, 2))


outcomes = TP_FP_FN_TN(Y_test, Y_prediction)
print(Y_prediction.shape == np.sum(outcomes))
outcomes

[ True]


array([[35744,    45],
       [  983,  1034]])

#### The next step is to calculate  Precision, Recall, F1 and F2 score 

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [42]:
def model_report(
    y_test,       # validated labels
    y_pred):      # predicted labels
    
    confusion_matr = np.transpose(TP_FP_FN_TN(y_test, y_pred))
    
    # [TP FP]
    # [FN TN]
    
    PRE = confusion_matr[0,0]/np.sum(confusion_matr[0,:])
    REC = confusion_matr[0,0]/np.sum(confusion_matr[:,0])
    ACC = np.trace(confusion_matr) / np.sum(confusion_matr)
    
    Fbeta = lambda beta: (
        (1 + beta ** 2) * (PRE * REC) / (((beta ** 2) * PRE) + REC)
    )
    
    print(confusion_matr)
    return {
        'PRE': PRE,
        'REC': REC,
        'F1S': Fbeta(1),
        'F2S': Fbeta(2),
        'ACC': ACC
    }
    

model_report(Y_test, Y_prediction)

[[35744   983]
 [   45  1034]]


{'PRE': 0.9732349497644784,
 'REC': 0.9987426304171673,
 'F1S': 0.9858238181918473,
 'F2S': 0.9935346864350717,
 'ACC': 0.9728085489075808}

In [43]:
# using sklearn package
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_prediction)

array([[35744,    45],
       [  983,  1034]], dtype=int64)

In [44]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_prediction))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99     35789
           1       0.96      0.51      0.67      2017

    accuracy                           0.97     37806
   macro avg       0.97      0.76      0.83     37806
weighted avg       0.97      0.97      0.97     37806



In [45]:
from sklearn.metrics import accuracy_score
A0 = accuracy_score(Y_test, Y_prediction)
A0

0.9728085489075808

Calculate these metrics for the vectorization created using count vectorizing and for tf-idf vectorization.  
Compare them. 

In [46]:
# calculation based on count matrices

X_train_q, X_test_q, Y_train_q, Y_test_q = train_test_split(
    X, Y, test_size = custom_test_size
)

X_train_q = X_train_q.reset_index(drop = True)
X_test_q = X_test_q.reset_index(drop = True)

Y_train_q = Y_train_q.reset_index(drop = True)
Y_test_q = Y_test_q.reset_index(drop = True)

X_train_concat_q =  X_train_q.str.join(' ')
X_test_concat_q = X_test_q.str.join(' ')

cnt_vect_model = CountVectorizer(vocabulary = cleaned_vocab)

X_train_q_sparse = cnt_vect_model.fit_transform(X_train_concat_q)
X_test_q_sparse = cnt_vect_model.transform(X_test_concat_q)

In [47]:
X_train_q_sparse

<113417x18705 sparse matrix of type '<class 'numpy.int64'>'
	with 1978963 stored elements in Compressed Sparse Row format>

In [48]:
linear_classifier_q = LogisticRegression(
    C = 1,
    random_state = 0
).fit(X_train_q_sparse, Y_train_q) # too bad, but let's try another model

c:\users\laplace-transform\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [49]:
# we're going to the forest!
# https://towardsdatascience.com/understanding-random-forest-58381e0602d2
# https://shorturl.at/fFOP4
# also, https://scikit-learn.org/stable/modules/tree.html#tree

In [55]:
rand_forest_model_q = RandomForestClassifier().fit(
    X_train_q_sparse, Y_train_q
)

In [56]:
Y_prediction_q = rand_forest_model.predict(X_test_q_sparse)

In [57]:
outcomes_q = confusion_matrix(Y_test_q, Y_prediction_q)
outcomes_q

array([[35587,   324],
       [  684,  1211]], dtype=int64)

In [58]:
AQ = accuracy_score(Y_test_q, Y_prediction_q)
AQ

0.9733375654657991

In [59]:
print(classification_report(Y_test_q, Y_prediction_q))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     35911
           1       0.79      0.64      0.71      1895

    accuracy                           0.97     37806
   macro avg       0.89      0.82      0.85     37806
weighted avg       0.97      0.97      0.97     37806



In [60]:
rand_forest_model_f = RandomForestClassifier().fit(
    X_train_tf_idf_sparse, Y_train
)

In [61]:
Y_prediction_f = rand_forest_model_f.predict(X_test_tf_idf_sparse)

In [62]:
outcomes_f = confusion_matrix(Y_test, Y_prediction_f)
outcomes_f

array([[35590,   199],
       [  775,  1242]], dtype=int64)

In [63]:
AF = accuracy_score(Y_test, Y_prediction_f)
AF

0.9742368936147702

In [64]:
print(classification_report(Y_test, Y_prediction_f))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     35789
           1       0.86      0.62      0.72      2017

    accuracy                           0.97     37806
   macro avg       0.92      0.81      0.85     37806
weighted avg       0.97      0.97      0.97     37806



In [65]:
# comparison: count vs tf-idf matrices. only accuracy comp

rf_q_less_rf_f = np.array([
  AQ < AF,
   # ...
])

rf_q_less_rf_f

array([True, Ellipsis], dtype=object)

In [66]:
A0 < AF # explanation below

True

In [67]:
from joblib import dump
#from joblib import load

dump(linear_classifier, 'rforest_tfidf_cls.joblib')

['rforest_tfidf_cls.joblib']

Пояснення:

### Conclusions and improvements 

For all of the vectorization pipelines we used all of the words, which were available in our dictionary, as experiment try to use the most meaningful words - select them using TF-IDF score. (for example for each text you can select not more than 10 words for vectorization, or less). 

Compare this approach with the first and second ones. Did your model improve? 



In [ ]:
# Another code for task above

X_full_concat = df.cleaned.str.join(' ')

pipeline_full = pipeline_global.fit(X_full_concat)

X_full_concat_tf_idf = pipeline_full.transform(X_full_concat)

In [ ]:
X_full_concat_tf_idf

In [ ]:
X_full_concat_tf_idf[0].nonzero()[1][::-1]

In [ ]:
to_filter = np.zeros(len(cleaned_vocab))

M = len(X_full_concat)

for j in range(M):
    
    to_add = np.zeros(len(cleaned_vocab))
    
    idx_non_zero = X_full_concat_tf_idf[j].nonzero()[1][::-1]
    
    np.put(to_add, idx_non_zero, np.ones(len(idx_non_zero)))
    
    to_filter += to_add

In [ ]:
print(len(X_full_concat_tf_idf.nonzero()[0]), sum(to_filter))

In [ ]:
len(to_filter[to_filter != 0])

In [ ]:
new_vocabulary = np.array(cleaned_vocab)[np.where(to_filter != 0)]
new_vocabulary

In [ ]:
X_train_u, X_test_u, Y_train_u, Y_test_u = train_test_split(
    X, Y, test_size = custom_test_size
)

X_train_u = X_train_u.reset_index(drop = True)
X_test_u = X_test_u.reset_index(drop = True)

Y_train_u = Y_train_u.reset_index(drop = True)
Y_test_u = Y_test_u.reset_index(drop = True)

In [ ]:
X_train_concat_u =  X_train_u.str.join(' ')
X_test_concat_u = X_test_u.str.join(' ')

X_train_tf_idf_sparse_upd = pipeline_global.fit_transform(X_train_concat_u)
X_test_tf_idf_sparse_upd = pipeline_global.transform(X_test_concat_u)

In [ ]:
linear_classifier.fit(X_train_tf_idf_sparse_upd, Y_train_u)

Y_prediction_upd = linear_classifier.predict(X_test_tf_idf_sparse)

In [ ]:
conf = confusion_matrix(Y_test, Y_prediction_upd)
conf

In [ ]:
# fix tp fp tn fn measures!

In [ ]:
A1 = accuracy_score(Y_test_u, Y_prediction_upd)
A1

In [ ]:
print(classification_report(Y_test_u, Y_prediction_upd))

In [ ]:
A1 > A0 # See!

In [ ]:
dump(linear_classifier, 'logistic_cls.joblib')

Висновки:

### Additionally, visualisations 

For now you have a vector for each word from your vocabulary. 
You have vectors with lenght > 18000, so the dimension of your space is more than 18000 - it's impossible to visualise it in 2d space. 

So try to research and look for algorithms which perform dimensionality reduction. (t-SNE, PCA) 
Try to visualise obtained vectors in a vectorspace, only subset from the vocabulary, don't plot all of the words. (100) 

Probably on this step you will realise how this type of vectorization using these techniques is not the best way to vectorize words. 

Please, analyse the obtained results and explain why visualisation looks like this. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Here we'll be using princomp method to make visualizations possible
# .. later, because having troubles with memory. Now using alternative for sparse matrices

# https://en.wikipedia.org/wiki/Singular_value_decomposition
# https://arxiv.org/pdf/0909.4061.pdf

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [ ]:
hot_vectors = np.array(list(vocabulary.values()))

pca_model = PCA(n_components = 3)
pca_model.fit(hot_vectors)

hot_vectors_transformed = pca_model.transform(hot_vectors)

In [ ]:
hot_vectors_transformed

In [ ]:
plt.figure(figsize = (12, 12))

def fast_scatter(
    data,
    N:     int = 100,
    i:     int = 0,
    j:     int = 0,
    fsize: tuple = (12, 12),
    s:     int = 16,
    col:   str = "red"):
    plt.figure(figsize = fsize)
    sns.scatterplot(
        x = hot_vectors_transformed[:500, i],
        y = hot_vectors_transformed[:500, j],
        s = 16,
        color = "red"
    )

# actually pretty similar for other pairs
fast_scatter(hot_vectors_transformed, 500, 1, 2)
fast_scatter(hot_vectors_transformed, 500, 0, 2)

In [ ]:
trunc_svd_model = TruncatedSVD(n_components = 7)

In [ ]:
t_non = df_non_toxic.cleaned.str.join(' ')
t_ins = df_insulting.cleaned.str.join(' ')

pipeline_n = Pipeline(
    [
        ('cnt-matr', CountVectorizer(vocabulary = cleaned_vocab)),
        ('tf-idf-sparse-matr', TfidfTransformer())
    ]
).fit(t_non)

pipeline_i = Pipeline(
    [
        ('cnt-matr', CountVectorizer(vocabulary = cleaned_vocab)),
        ('tf-idf-sparse-matr', TfidfTransformer())
    ]
).fit(t_ins)

non_toxic_edited = pipeline_n.transform(t_non)
insulting_edited = pipeline_i.transform(t_ins)

In [ ]:
final_n = trunc_svd_model.fit_transform(non_toxic_edited)

print(
    trunc_svd_model.explained_variance_ratio_
)

final_i = trunc_svd_model.fit_transform(insulting_edited)

print(
    trunc_svd_model.explained_variance_ratio_
)

In [ ]:
lim_n = 500

n, m = 4, 6

transformed_non_toxic = pd.DataFrame(final_n[:lim_n, (n,m)], columns = ['X', 'Y'])
transformed_insulting = pd.DataFrame(final_i[:lim_n, (n,m)], columns = ['X', 'Y'])

In [ ]:
plt.figure(figsize = (12, 12))

sns.scatterplot(
    x = 'X',
    y = 'Y',
    data = pd.concat([transformed_non_toxic.assign(set_type='non-toxic'), transformed_insulting.assign(set_type='insulting')]),
    s = 12, hue = 'set_type'
)

In [ ]:
# TODO:
# tf-idf manually
# quantity-based matrix into model, then compare results